# Ecommerce Runner

This notebook is the Jupyter equivalent of the `runner.py` script.

In [1]:
import json
import logging
import os
import sys
from datetime import datetime, timezone
from pathlib import Path

from dotenv import load_dotenv
from rich.pretty import pprint

from graphiti_core import Graphiti
from graphiti_core.edges import EntityEdge
from graphiti_core.llm_client.anthropic_client import AnthropicClient
from graphiti_core.nodes import EpisodeType
from graphiti_core.utils.bulk_utils import RawEpisode
from graphiti_core.utils.maintenance.graph_data_operations import clear_data

load_dotenv()

True

In [2]:
neo4j_uri = os.environ.get('NEO4J_URI', 'bolt://localhost:7687')
neo4j_user = os.environ.get('NEO4J_USER', 'neo4j')
neo4j_password = os.environ.get('NEO4J_PASSWORD', 'password')

In [3]:
def setup_logging():
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    return logger


logger = setup_logging()

In [4]:
shoe_conversation_1 = [
    "SalesBot (2024-07-30T00:00:00Z): Hi, I'm ManyBirds Assistant! How can I help you today?",
    "John (2024-07-30T00:01:00Z): Hi, I'm looking for a new pair of shoes.",
    'SalesBot (2024-07-30T00:02:00Z): Of course! What kind of material are you looking for?',
    "John (2024-07-30T00:03:00Z): I'm allergic to wool. Also, I'm a size 10 if that helps?",
    "SalesBot (2024-07-30T00:04:00Z): We have just what you are looking for, how do you like our Men's Couriers. They have a retro silhouette look and from cotton. How about them in Basin Blue?",
    "John (2024-07-30T00:05:00Z): Blue is great! Love the look. I'll take them.",
]

shoe_conversation_2 = [
    'SalesBot (2024-08-20T00:00:00Z): Hi John, how can I assist you today?',
    "John (2024-08-20T00:01:00Z): Hi, I need to return the Men's Couriers I bought recently. They're too tight for my wide feet. Hahaha.",
    "SalesBot (2024-08-20T00:02:00Z): I'm sorry to hear that. We can process the return for you.",
]

In [5]:
async def add_messages(client: Graphiti, messages: list[str], prefix: str = 'Message'):
    for i, message in enumerate(messages):
        await client.add_episode(
            name=f'{prefix}-{i}',
            episode_body=message,
            source=EpisodeType.message,
            reference_time=datetime.now(timezone.utc),
            source_description='Shoe conversation',
        )

In [6]:
async def ingest_products_data(client: Graphiti):
    script_dir = Path.cwd().parent
    json_file_path = script_dir / 'data' / 'manybirds_products.json'

    with open(json_file_path) as file:
        products = json.load(file)['products']

    episodes: list[RawEpisode] = [
        RawEpisode(
            name=product.get('title', f'Product {i}'),
            content=str({k: v for k, v in product.items() if k != 'images'}),
            source_description='ManyBirds products',
            source=EpisodeType.json,
            reference_time=datetime.now(timezone.utc),
        )
        for i, product in enumerate(products)
    ]

    await client.add_episode_bulk(episodes)

In [7]:
def pretty_print(entity: EntityEdge | list[EntityEdge]):
    if isinstance(entity, EntityEdge):
        data = {k: v for k, v in entity.model_dump().items() if k != 'fact_embedding'}
    elif isinstance(entity, list):
        data = [{k: v for k, v in e.model_dump().items() if k != 'fact_embedding'} for e in entity]
    else:
        pprint(entity)
        return
    pprint(data)

In [8]:
llm_client = AnthropicClient(cache=False)

client = Graphiti(
    neo4j_uri,
    neo4j_user,
    neo4j_password,
    llm_client=llm_client,
)

In [9]:
await clear_data(client.driver)
await client.build_indices_and_constraints()
await ingest_products_data(client)

neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX entity_uuid IF NOT EXISTS FOR (e:Entity) ON (e.uuid)` has no effect.} {description: `RANGE INDEX entity_uuid FOR (e:Entity) ON (e.uuid)` already exists.} {position: None} for query: 'CREATE INDEX entity_uuid IF NOT EXISTS FOR (n:Entity) ON (n.uuid)'
neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX name_entity_index IF NOT EXISTS FOR (e:Entity) ON (e.name)` has no effect.} {description: `RANGE INDEX name_entity_index FOR (e:Entity) ON (e.name)` already exists.} {position: None} for query: 'CREATE INDEX name_entity_index IF NOT EXISTS FOR (n:Entity) ON (n.name)'
neo4j.notifications - INFO - Received notification from DBMS ser

In [10]:
await client.add_episode(
    name='Inventory management 0',
    episode_body=('All Tinybirds Wool Runners styles are out of stock until December 25th 2024'),
    source=EpisodeType.text,
    reference_time=datetime.now(timezone.utc),
    source_description='Inventory Management Bot',
)

httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
graphiti_core.utils.maintenance.node_operations - INFO - Extracted new nodes: [{'name': 'AI Assistant', 'labels': ['Entity', 'Speaker'], 'summary': 'AI providing information about product availability'}, {'name': 'Tinybirds Wool Runners', 'labels': ['Entity', 'Product'], 'summary': "Children's eco-friendly sneakers made with ZQ Merino Wool"}] in 2495.445966720581 ms
graphiti_core.utils.maintenance.node_operations - INFO - Created new node: AI Assistant (UUID: a06d832a07fc403f8e43df6b2b650f1a)
graphiti_core.utils.maintenance.node_operations - INFO - Created new node: Tinybirds Wool Runners (UUID: d3238edc2de14a23bf63b4e0ff751d8c)
graphiti_core.graphiti - INFO - Extracted nodes: [('AI Assistant', 'a06d832a07fc403f8e43df6b2b650f1a'), ('Tinybirds Wool Runners', 'd3238edc2de14a23bf63b4e0ff751d8c')]
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
httpx - INFO - H

In [11]:
r = await client.search('Which products are out of stock?')

pretty_print(r[0])

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query Which products are out of stock? in 206.62617683410645 ms


{
│   'uuid': '150fce971e43402582df51d83e09dddf',
│   'source_node_uuid': 'a06d832a07fc403f8e43df6b2b650f1a',
│   'target_node_uuid': '138a288fc46f40a18623ccf970d49813',
│   'created_at': datetime.datetime(2024, 8, 31, 11, 34, 12, 9589),
│   'name': 'PROVIDES_AVAILABILITY_INFO',
│   'fact': 'AI Assistant informs that all TinyBirds Wool Runners styles are out of stock until December 25th 2024',
│   'episodes': ['1de5e192b93149b5a11ede5667d99a40'],
│   'expired_at': datetime.datetime(2024, 8, 31, 11, 34, 16, 47041),
│   'valid_at': None,
│   'invalid_at': datetime.datetime(2024, 12, 25, 0, 0, tzinfo=<UTC>)
}

In [12]:
await add_messages(client, shoe_conversation_1, prefix='conversation-1')

httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
graphiti_core.utils.maintenance.node_operations - INFO - Extracted new nodes: [{'name': 'SalesBot', 'labels': ['Entity', 'Speaker', 'AI'], 'summary': 'AI assistant for ManyBirds, designed to help customers'}, {'name': 'ManyBirds', 'labels': ['Entity', 'Company'], 'summary': 'Company that the SalesBot represents and assists customers for'}] in 2248.044967651367 ms
graphiti_core.utils.maintenance.node_operations - INFO - Created new node: SalesBot (UUID: d362076a1e584227bcf51239914e39ad)
graphiti_core.utils.maintenance.node_operations - INFO - Created new node: ManyBirds (UUID: cf011889a3ab400aa6d4efa2a5bbf70b)
graphiti_core.graphiti - INFO - Extracted nodes: [('SalesBot', 'd362076a1e584227bcf51239914e39ad'), ('ManyBirds', 'cf011889a3ab400aa6d4efa2a5bbf70b')]
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.nodes - INFO - embedded SalesBot in 0.1

In [13]:
r = await client.search("What is John's shoe size?", num_results=2)

pretty_print(r)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query What is John's shoe size? in 204.0848731994629 ms


[
│   {
│   │   'uuid': '6a19ae37d5074d808d4f951ab347e2b1',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'fcea4a4539244cd28aac1bb11def0cab',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 35, 44, 738829),
│   │   'name': 'HAS_SHOE_SIZE',
│   │   'fact': "John's shoe size is 10",
│   │   'episodes': ['6b41a387ca504a2686b636a20b5673a3'],
│   │   'expired_at': None,
│   │   'valid_at': datetime.datetime(2024, 7, 30, 0, 3, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '0c150ca1debc423eb7e3bd535413c782',
│   │   'source_node_uuid': '0e96a1b72fe145a79ec2b36842ac6fd9',
│   │   'target_node_uuid': '0e96a1b72fe145a79ec2b36842ac6fd9',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 33, 39, 424173),
│   │   'name': 'IS_VARIANT_OF',
│   │   'fact': "The Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a specific variant of the SuperLight Wool Runner line",
│   │   'episodes': ['4a302ac072c94f9da876535b1130e03d'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   }
]

In [14]:
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF

nl = await client._search('John', NODE_HYBRID_SEARCH_RRF)

pretty_print(nl[0])

john_uuid = nl[0].uuid

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search_utils - INFO - Found relevant nodes: {'c4091c3ffc814f2c9017304361898585', '95066726921c4e5883a86d8095cd7e0a', 'ccd7590b3601440f9ae816507da79130', 'fcea4a4539244cd28aac1bb11def0cab', '8169219a1c564a53a7201bf215bd45f8', 'b30e3ba27aa14f88895156331a435237', 'c4efdae7ab9240fd8b8f59ac741a19bf'} in 8.331060409545898 ms


EntityNode(
│   uuid='c4091c3ffc814f2c9017304361898585',
│   name='John',
│   labels=['Entity'],
│   created_at=datetime.datetime(2024, 8, 31, 11, 34, 52, 870658),
│   name_embedding=None,
│   summary='Customer looking for a new pair of shoes'
)

In [15]:
r = await client.search('Can John wear ManyBirds Wool Runners?', num_results=3)

print('-' * 100)
print('Standard Reciprocal Rank Fusion Reranking')
print('-' * 100)
for record in r:
    print(record.fact)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query Can John wear ManyBirds Wool Runners? in 252.65789031982422 ms
----------------------------------------------------------------------------------------------------
Standard Reciprocal Rank Fusion Reranking
----------------------------------------------------------------------------------------------------
TinyBirds Wool Runners are available in Natural Black color
The Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a specific variant of the SuperLight Wool Runner line
John is allergic to wool


In [16]:
r = await client.search(
    'Can John wear ManyBirds Wool Runners?', center_node_uuid=john_uuid, num_results=3
)

print('-' * 100)
print("Node Distance Reranking from 'John' node")
print('-' * 100)
for record in r:
    print(record.fact)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query Can John wear ManyBirds Wool Runners? in 310.61410903930664 ms
----------------------------------------------------------------------------------------------------
Node Distance Reranking from 'John' node
----------------------------------------------------------------------------------------------------
TinyBirds Wool Runners are available in Natural Black color
The Men's SuperLight Wool Runners - Dark Grey (Medium Grey Sole) is a specific variant of the SuperLight Wool Runner line
John is allergic to wool


In [17]:
await add_messages(client, shoe_conversation_2, prefix='conversation-2')

httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
graphiti_core.utils.maintenance.node_operations - INFO - Extracted new nodes: [{'name': 'SalesBot', 'labels': ['Entity', 'Speaker', 'AI'], 'summary': 'AI sales assistant engaging with the customer'}, {'name': 'John', 'labels': ['Entity', 'Customer'], 'summary': 'Customer being addressed by the SalesBot'}] in 1890.765905380249 ms
graphiti_core.utils.maintenance.node_operations - INFO - Created new node: SalesBot (UUID: c807d7ac10014a6faf0c5e4c9dbc3eac)
graphiti_core.utils.maintenance.node_operations - INFO - Created new node: John (UUID: cbef7be8d9a5481dbe2f56be97d0e462)
graphiti_core.graphiti - INFO - Extracted nodes: [('SalesBot', 'c807d7ac10014a6faf0c5e4c9dbc3eac'), ('John', 'cbef7be8d9a5481dbe2f56be97d0e462')]
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_

In [18]:
r = await client.search('What shoes has John purchased?', center_node_uuid=john_uuid, num_results=3)

pretty_print(r)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query What shoes has John purchased? in 215.75593948364258 ms


[
│   {
│   │   'uuid': '199ec767d52c47d2a5965f3197b1c4d2',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'b30e3ba27aa14f88895156331a435237',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 36, 42, 827088),
│   │   'name': 'PURCHASES',
│   │   'fact': "John purchased the Men's Couriers shoes but later decided to return them due to discomfort caused by his wide feet",
│   │   'episodes': ['4c8afb4aa1b446899a85249df475bc66'],
│   │   'expired_at': datetime.datetime(2024, 8, 31, 11, 38, 14, 818497),
│   │   'valid_at': datetime.datetime(2024, 7, 30, 0, 5, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '2a9cf189e19649c19ec127c4024cfe51',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '77f8b23b74014a7f85fffa0067dbf815',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 34, 57, 412667),
│   │   'name': 'INTERESTED_IN',
│   │   'fact': 'John is looking for a new pair of shoes',
│   │   'episodes': ['c2ebc79d2a204efb845be84b6dbf69d7'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': 'aa657e8bcb9446e19552f99a1c2299d8',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '8b43988e689b437095c7e75aa1044490',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 37, 39, 665400),
│   │   'name': 'HAS_CHARACTERISTIC',
│   │   'fact': 'John has wide feet',
│   │   'episodes': ['37c0e9ecaa424caea59854d1d8c2c756'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   }
]

In [19]:
r = await client.search('What shoes has John purchased?', center_node_uuid=john_uuid, num_results=5)

pretty_print(r)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query What shoes has John purchased? in 231.48012161254883 ms


[
│   {
│   │   'uuid': '199ec767d52c47d2a5965f3197b1c4d2',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'b30e3ba27aa14f88895156331a435237',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 36, 42, 827088),
│   │   'name': 'PURCHASES',
│   │   'fact': "John purchased the Men's Couriers shoes but later decided to return them due to discomfort caused by his wide feet",
│   │   'episodes': ['4c8afb4aa1b446899a85249df475bc66'],
│   │   'expired_at': datetime.datetime(2024, 8, 31, 11, 38, 14, 818497),
│   │   'valid_at': datetime.datetime(2024, 7, 30, 0, 5, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '2a9cf189e19649c19ec127c4024cfe51',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '77f8b23b74014a7f85fffa0067dbf815',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 34, 57, 412667),
│   │   'name': 'INTERESTED_IN',
│   │   'fact': 'John is looking for a new pair of shoes',
│   │   'episodes': ['c2ebc79d2a204efb845be84b6dbf69d7'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': 'aa657e8bcb9446e19552f99a1c2299d8',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '8b43988e689b437095c7e75aa1044490',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 37, 39, 665400),
│   │   'name': 'HAS_CHARACTERISTIC',
│   │   'fact': 'John has wide feet',
│   │   'episodes': ['37c0e9ecaa424caea59854d1d8c2c756'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': 'df1d2e82a40e40e1b3734c2298774a6b',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '588989497641456fb33243f035731f98',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 36, 42, 828745),
│   │   'name': 'LIKES',
│   │   'fact': 'John expresses that he likes the Basin Blue color for the shoes',
│   │   'episodes': ['4c8afb4aa1b446899a85249df475bc66'],
│   │   'expired_at': None,
│   │   'valid_at': datetime.datetime(2024, 7, 30, 0, 5, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '6a19ae37d5074d808d4f951ab347e2b1',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'fcea4a4539244cd28aac1bb11def0cab',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 35, 44, 738829),
│   │   'name': 'HAS_SHOE_SIZE',
│   │   'fact': "John's shoe size is 10",
│   │   'episodes': ['6b41a387ca504a2686b636a20b5673a3'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   }
]

In [20]:
r = await client.search('Who is John?', num_results=5)

pretty_print(r)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query Who is John? in 211.70878410339355 ms


[
│   {
│   │   'uuid': 'e4cd07dfddc84072985aa8cf4e1dc01b',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'ccd7590b3601440f9ae816507da79130',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 35, 44, 738205),
│   │   'name': 'IS_ALLERGIC_TO',
│   │   'fact': 'John is allergic to wool',
│   │   'episodes': ['6b41a387ca504a2686b636a20b5673a3'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': 'aa657e8bcb9446e19552f99a1c2299d8',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '8b43988e689b437095c7e75aa1044490',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 37, 39, 665400),
│   │   'name': 'HAS_CHARACTERISTIC',
│   │   'fact': 'John has wide feet',
│   │   'episodes': ['37c0e9ecaa424caea59854d1d8c2c756'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '6a19ae37d5074d808d4f951ab347e2b1',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'fcea4a4539244cd28aac1bb11def0cab',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 35, 44, 738829),
│   │   'name': 'HAS_SHOE_SIZE',
│   │   'fact': "John's shoe size is 10",
│   │   'episodes': ['6b41a387ca504a2686b636a20b5673a3'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '518d5ef539004ceca7b9b9a750e22bd4',
│   │   'source_node_uuid': 'd362076a1e584227bcf51239914e39ad',
│   │   'target_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 37, 15, 423989),
│   │   'name': 'ASSISTS',
│   │   'fact': 'SalesBot offers assistance to John',
│   │   'episodes': ['7087342bfe86423bb702060fa9cc612b'],
│   │   'expired_at': None,
│   │   'valid_at': datetime.datetime(2024, 8, 20, 0, 2, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '2a9cf189e19649c19ec127c4024cfe51',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '77f8b23b74014a7f85fffa0067dbf815',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 34, 57, 412667),
│   │   'name': 'INTERESTED_IN',
│   │   'fact': 'John is looking for a new pair of shoes',
│   │   'episodes': ['c2ebc79d2a204efb845be84b6dbf69d7'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   }
]

In [21]:
r = await client.search(
    'What did John do about his discomfort with the Mens Couriers shoes', num_results=5
)

pretty_print(r)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
graphiti_core.search.search - INFO - search returned context for query What did John do about his discomfort with the Mens Couriers shoes in 215.81482887268066 ms


[
│   {
│   │   'uuid': '1055fb8279af4c4c8c3fb78350d610d0',
│   │   'source_node_uuid': '8b43988e689b437095c7e75aa1044490',
│   │   'target_node_uuid': 'b30e3ba27aa14f88895156331a435237',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 37, 39, 664102),
│   │   'name': 'CAUSES_DISCOMFORT',
│   │   'fact': "John's wide feet cause discomfort with the Men's Couriers shoes",
│   │   'episodes': ['37c0e9ecaa424caea59854d1d8c2c756'],
│   │   'expired_at': None,
│   │   'valid_at': datetime.datetime(2024, 8, 20, 0, 1, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '199ec767d52c47d2a5965f3197b1c4d2',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': 'b30e3ba27aa14f88895156331a435237',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 36, 42, 827088),
│   │   'name': 'PURCHASES',
│   │   'fact': "John purchased the Men's Couriers shoes but later decided to return them due to discomfort caused by his wide feet",
│   │   'episodes': ['4c8afb4aa1b446899a85249df475bc66'],
│   │   'expired_at': datetime.datetime(2024, 8, 31, 11, 38, 14, 818497),
│   │   'valid_at': datetime.datetime(2024, 7, 30, 0, 5, tzinfo=<UTC>),
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '2a9cf189e19649c19ec127c4024cfe51',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '77f8b23b74014a7f85fffa0067dbf815',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 34, 57, 412667),
│   │   'name': 'INTERESTED_IN',
│   │   'fact': 'John is looking for a new pair of shoes',
│   │   'episodes': ['c2ebc79d2a204efb845be84b6dbf69d7'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': '4721330c8f2b45e69e07f520773f8794',
│   │   'source_node_uuid': 'd362076a1e584227bcf51239914e39ad',
│   │   'target_node_uuid': 'ed9688ba1e9940ff87d3e26bcf5d7ae4',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 36, 12, 540437),
│   │   'name': 'RECOMMENDS',
│   │   'fact': "SalesBot recommends Men's Couriers shoes to the customer",
│   │   'episodes': ['e7c29d5d38854cac801bc07d236240a8'],
│   │   'expired_at': None,
│   │   'valid_at': None,
│   │   'invalid_at': None
│   },
│   {
│   │   'uuid': 'df1d2e82a40e40e1b3734c2298774a6b',
│   │   'source_node_uuid': 'c4091c3ffc814f2c9017304361898585',
│   │   'target_node_uuid': '588989497641456fb33243f035731f98',
│   │   'created_at': datetime.datetime(2024, 8, 31, 11, 36, 42, 828745),
│   │   'name': 'LIKES',
│   │   'fact': 'John expresses that he likes the Basin Blue color for the shoes',
│   │   'episodes': ['4c8afb4aa1b446899a85249df475bc66'],
│   │   'expired_at': None,
│   │   'valid_at': datetime.datetime(2024, 7, 30, 0, 5, tzinfo=<UTC>),
│   │   'invalid_at': None
│   }
]